In [ ]:
import requests 
import pandas as pd
import time
import json
from datetime import date,datetime,timedelta

In [ ]:
#Declare file config
file_path = '../credentials/config.json'

#Open and read Json file
with open(file_path, "r") as file:
    config_data = json.load(file)

In [ ]:
current_date = date.today()
current_date = current_date.strftime('%d/%m/%Y')
print("Current Date:", current_date)


In [ ]:
url = "https://fc-data.ssi.com.vn/api/v2/Market/AccessToken"

payload = {
    "consumerID": config_data["consumerID"],
    "consumerSecret": config_data["consumerSecret"]
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    access_token = response.json()['data'].get("accessToken")
else:
    print("Failed to get access token. Status Code:", response.status_code)
    print("Response Body:", response.text)
    exit()

In [ ]:
start_date = (date.today() - timedelta(days = 1)).strftime('%d/%m/%Y')
end_date = (date.today() - timedelta(days = 1)).strftime('%d/%m/%Y')

In [ ]:
print(start_date)
print(end_date)

In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/IndexList"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "pageIndex": 1,
    "pageSize": 1000
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Extract data from the response JSON
data = response.json().get('data', [])

index_list = []
for i in range(len(data)):
    index_list.append(data[i]['IndexCode'])

print(index_list)

In [ ]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/DailyIndex"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

df = pd.DataFrame()

for index in index_list:
    for page_index in range(1, 10):
        params = {
            "indexId": index,
            "fromDate":start_date,
            "toDate": end_date,
            "pageIndex": page_index,
            "pageSize": 1000,
            "ascending": False
            }
        response = requests.get(url, headers=headers, params=params)
        data = response.json().get('data', [])
        df_response = pd.DataFrame(data)
        df = pd.concat([df, df_response], ignore_index=True)
        time.sleep(1)
        if len(df_response) == 0:
            break

In [ ]:
display(df)